In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

model_data = pd.read_parquet('Leobrix_Release_BBILA_model_Data.parquet.gzip')
#Merge with new one banking features
sal_data = pd.read_csv('stated_salary_vs_actual_salary.csv')
sal_data = sal_data[['latest_feature_id','diff_btw_stated_salary_vs_actual_salary']]
model_data = pd.merge(model_data, sal_data, how='left',on='latest_feature_id')
#Merge with new pg features
pg_data= pd.read_parquet('s3://sandbox-payments-data-transfer/cashloans/Uptop_Darwin_24/feat_old_pipeline/pg_features/pg_features.parquet')
model_data = pd.merge(model_data, pg_data, how='left',on='latest_feature_id')
print('********* Data loaded ********* ')

********* Required Libraries Imported *********
********* Data loaded ********* 


In [2]:
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)
# Create one feature from inututive bureau features
data['num_CONSUMER LOAN_enq_last_6M_12M'] = (data['num_CONSUMER LOAN_enq_last_6M']+data['num_CONSUMER LOAN_enq_last_36M'])/2

In [3]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(39010, 2852) (12128, 2852)


In [4]:
features_list =[
    
#Bureau Intutive features
'Num_Enq_all_loans_last_9M_12M',
'Utilization_all',
'total_emi',
'Utilization_CREDIT CARD_opened_last_36M',
'Utilization_PERSONAL LOAN_opened_last_9M',
'num_CREDIT CARD_enq_last_6M',
'num_CONSUMER LOAN_enq_last_36M',
'Num_of_PERSONAL LOAN_opened_last6M_12M',
'highest_sanction_amount_credit_cards',
'num_CONSUMER LOAN_enq_last_6M',
'num_PL_LE_1_Lac_opened_in_last_12M',
'Total_Sanction_amt_PERSONAL LOAN_Opened_last_24M',
'Age_of_oldest_open_account',
    
#Banking Intutive features

'num_credit_gt_salary',    
'upi_trans_count',
'ratio_balance_after_7_days_max_credit_M3',    
'ratio_max_credit_to_total_credit_M2',
'spend_vs_salary_ratio',
'bounced_transactions_count',
'ratio_total_debit_to_max_credit_M2_M3',
'min_balance_last_3_months',
'min_balance_M2',
'days_balance_lt_5000_M3',
'days_balance_lt_10000_M3',
'Max_Credit_Avg_M1_M2_M3',
'Max_Credit_Diff_M1_M2_M3',
'amount_debits_last_2_months',
'sum_insurance_debit',
'avg_num_debits_last_2_months',
'upi_trans_debit_sum',
'cash_withdrawal_sum',
    
#new banking features
'diff_btw_stated_salary_vs_actual_salary',
'balance_6th_day_M2',
'ratio_6th_day_balance_to_max_credit_M3',

#Loan application Features

'deviation_from_median_income/standard_deviation_income#last_365_Days',
# 'yearOfBirth_mean',
'loan_amount_to_income#last_365_Days',


# # Reconcilated Features
'percentage_ever_delinquent', 
'months_since_most_recent_delinquency_installment', 
'percentage_utilization_gt_75',
'Num_of_installment_loan_opened_last_6M',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'month_year',
'90_in_24mob'
              
]




In [5]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_2802340/282645959.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_2802340/282645959.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [7]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23406, 42), (15604, 42), (23406,), (15604,))

In [8]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(18724, 42) (4682, 42) (15604, 42)


In [9]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [47]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,

    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'min_child_weight':25,

    #'lambda': 2.6,
    'lambda': 5.5,
    'alpha':5.0,
    'gamma':4.5,
    'scale_pos_weight':3.8,
    'random_state': 24
    
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 700,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.60795	eval-auc:0.61009
[10]	train-auc:0.64601	eval-auc:0.63558
[20]	train-auc:0.64559	eval-auc:0.63670
[30]	train-auc:0.64713	eval-auc:0.63847
[40]	train-auc:0.64768	eval-auc:0.63892
[50]	train-auc:0.64925	eval-auc:0.64051
[60]	train-auc:0.65025	eval-auc:0.64206
[70]	train-auc:0.65138	eval-auc:0.64351
[80]	train-auc:0.65185	eval-auc:0.64432
[90]	train-auc:0.65267	eval-auc:0.64523
[100]	train-auc:0.65399	eval-auc:0.64676
[110]	train-auc:0.65456	eval-auc:0.64678
[120]	train-auc:0.65562	eval-auc:0.64752
[130]	train-auc:0.65647	eval-auc:0.64802
[140]	train-auc:0.65789	eval-auc:0.64926
[150]	train-auc:0.65858	eval-auc:0.65003
[160]	train-auc:0.65954	eval-auc:0.65078
[170]	train-auc:0.66065	eval-auc:0.65186
[180]	train-auc:0.66174	eval-auc:0.65274
[190]	train-auc:0.66285	eval-auc:0.65319
[200]	train-auc:0.66408	eval-auc:0.65404
[210]	train-auc:0.66505	eval-auc:0.65461
[220]	train-auc:0.66612	eval-auc:0.65526
[230]	train-auc:0.66754	eval-auc:0.65598
[240]	train-auc:0.66834	eva

In [48]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.69, Gini: 0.38, KS: 0.28
Eval AUC: 0.66, Gini: 0.33, KS: 0.25
Test AUC: 0.67, Gini: 0.33, KS: 0.24
OOT AUC: 0.67, Gini: 0.34, KS: 0.26


In [49]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.392952         0.641680        0.822942        0.576139   1873   
9          0.262286         0.540561        0.576113        0.509236   1872   
8          0.200321         0.485496        0.509233        0.463684   1872   
7          0.197544         0.444437        0.463676        0.426713   1873   
6          0.159722         0.410526        0.426705        0.394769   1872   
5          0.144765         0.380582        0.394760        0.366969   1872   
4          0.114255         0.353525        0.366963        0.340014   1873   
3          0.096740         0.325081        0.340004        0.310305   1871   
2          0.067806         0.295386        0.310302        0.277792   1873   
1          0.045916         0.245614        0.277786        0.150293   1873   

        label_good  label_bad  
Decile                         
10            1137        736  
9             1381        491  
8             1497        375  
7             1503        370  
6             1573        299  
5             1601        271  
4             1659        214  
3             1690        181  
2             1746        127  
1             1787         86

In [50]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.367072         0.640149        0.813193        0.574735   1561   
9          0.260897         0.539311        0.574618        0.508610   1560   
8          0.212179         0.484713        0.508607        0.463096   1560   
7          0.173607         0.444581        0.463042        0.426869   1561   
6          0.173077         0.410002        0.426839        0.394536   1560   
5          0.135897         0.380616        0.394417        0.367280   1560   
4          0.122998         0.353778        0.367246        0.340021   1561   
3          0.107051         0.325549        0.340012        0.311224   1560   
2          0.098718         0.295492        0.311196        0.277224   1560   
1          0.057655         0.244846        0.277213        0.149240   1561   

        label_good  label_bad  
Decile                         
10             988        573  
9             1153        407  
8             1229        331  
7             1290        271  
6             1290        270  
5             1348        212  
4             1369        192  
3             1393        167  
2             1406        154  
1             1471         90

In [51]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.375103         0.637420        0.823880        0.572536   1213   
9          0.238252         0.536223        0.572494        0.506105   1213   
8          0.225886         0.482625        0.506083        0.460976   1213   
7          0.204620         0.442553        0.460967        0.424099   1212   
6          0.156636         0.408680        0.424021        0.392495   1213   
5          0.153339         0.378683        0.392490        0.365874   1213   
4          0.122112         0.352986        0.365811        0.339109   1212   
3          0.102226         0.325383        0.339105        0.311932   1213   
2          0.082440         0.297946        0.311927        0.281392   1213   
1          0.061830         0.248884        0.281345        0.164319   1213   

        label_good  label_bad  
Decile                         
10             758        455  
9              924        289  
8              939        274  
7              964        248  
6             1023        190  
5             1027        186  
4             1064        148  
3             1089        124  
2             1113        100  
1             1138         75

In [52]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
8                highest_sanction_amount_credit_cards       179.0
12                         Age_of_oldest_open_account       149.0
0                       Num_Enq_all_loans_last_9M_12M       141.0
41                          Num_Enq_non_BANK_last_12M       118.0
30                                cash_withdrawal_sum       113.0
34  deviation_from_median_income/standard_deviatio...        93.0
4            Utilization_PERSONAL LOAN_opened_last_9M        91.0
36                         percentage_ever_delinquent        86.0
1                                     Utilization_all        73.0
18                         bounced_transactions_count        71.0
3             Utilization_CREDIT CARD_opened_last_36M        69.0
9                       num_CONSUMER LOAN_enq_last_6M        61.0
14                                    upi_trans_count        56.0
40                           No of enq in past 90days        56.0
5         

In [53]:
feature_importance_df

Feature  Importance
8                highest_sanction_amount_credit_cards       179.0
12                         Age_of_oldest_open_account       149.0
0                       Num_Enq_all_loans_last_9M_12M       141.0
41                          Num_Enq_non_BANK_last_12M       118.0
30                                cash_withdrawal_sum       113.0
34  deviation_from_median_income/standard_deviatio...        93.0
4            Utilization_PERSONAL LOAN_opened_last_9M        91.0
36                         percentage_ever_delinquent        86.0
1                                     Utilization_all        73.0
18                         bounced_transactions_count        71.0
3             Utilization_CREDIT CARD_opened_last_36M        69.0
9                       num_CONSUMER LOAN_enq_last_6M        61.0
14                                    upi_trans_count        56.0
40                           No of enq in past 90days        56.0
5                         num_CREDIT CARD_enq_last_6M        47.0
21                                     min_balance_M2        45.0
7              Num_of_PERSONAL LOAN_opened_last6M_12M        43.0
27                                sum_insurance_debit        40.0
37   months_since_most_recent_delinquency_installment        39.0
35                loan_amount_to_income#last_365_Days        35.0
22                            days_balance_lt_5000_M3        33.0
39             Num_of_installment_loan_opened_last_6M        33.0
10                 num_PL_LE_1_Lac_opened_in_last_12M        30.0
32                                 balance_6th_day_M2        26.0
6                      num_CONSUMER LOAN_enq_last_36M        23.0
2                                           total_emi        23.0
33             ratio_6th_day_balance_to_max_credit_M3        22.0
13                               num_credit_gt_salary        21.0
20                          min_balance_last_3_months        19.0
38                       percentage_utilization_gt_75        18.0
17                              spend_vs_salary_ratio        18.0
23                           days_balance_lt_10000_M3        16.0
15           ratio_balance_after_7_days_max_credit_M3        16.0
28                       avg_num_debits_last_2_months        15.0
11   Total_Sanction_amt_PERSONAL LOAN_Opened_last_24M        14.0
24                            Max_Credit_Avg_M1_M2_M3        12.0
16                ratio_max_credit_to_total_credit_M2        11.0
25                           Max_Credit_Diff_M1_M2_M3        10.0
29                                upi_trans_debit_sum        10.0
26                        amount_debits_last_2_months         7.0
31            diff_btw_stated_salary_vs_actual_salary         5.0
19              ratio_total_debit_to_max_credit_M2_M3         3.0

In [54]:
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BBLA_Train_Pred_9024.csv',index=False)
Eval.to_csv('LeoPayu_BBLA_Eval_Pred_9024.csv',index=False)
Test.to_csv('LeoPayu_BBLA_Test_Pred_9024.csv',index=False)
Oot.to_csv('LeoPayu_BBLA_Oot_Pred_9024.csv',index=False)


In [55]:
xgb_model.save_model('LeoPayu_BBLA_90_in_24mob.json')
print('Model Saved')

Model Saved
